## project outlines

- In this project we are going to scrape 'http://github.com/topics'.
- we are going to scrape github's entire list of topics
- For each topic we are going to get a list of upto 7 pages usernames,reponames and their stars
- This project is divided into two sections
- section 1 scrapes the entire topics and creates a dataframe consisting of topicname,description and url
- section 2 scrapes the contents of each topic by taking topic url from the dataframe.

## Section 1

- we are scraping a list of topics of upto 5 pages
- we are creating three arrays named topic_titles,topic_desc(for descriptions) and topic_url(for urls)

In [74]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [75]:
#Block-10
#takes lists of titles,desc,urls and returns a dataframe for the same
def convert_to_dataframe(topic_titles,topic_desc,
                            topic_url):
    
    df_dict = {
        'title':topic_titles,
        'desc': topic_desc,
        'url': topic_url
    }
    df = pd.DataFrame(df_dict)
    return df

In [76]:
#Block-9
#Takes in tags containing url and a list, extracts url from tags and returns list of urls of topics
def extract_url(url_tags,
                topic_url):
    
    
    
    for tp_url in url_tags:
        topic_url.append('http://github.com'+tp_url['href'])
    
    
    

In [77]:
# This functions scrapes description from the topic pages
'''
It takes p tags in whic descripion is present and titles_desc list as input,Extracts description
and appends to list.
'''
#Block-8
def extract_description(desc_tags,
                       topic_desc):
    
        for tag in desc_tags:
            topic_desc.append(tag.text.strip())

In [78]:
# This functions scrapes titles from the topic pages
#It takes tags where topic title present and titles list as input,Extracts titles and appends to list.

#Block-7
def extract_titles(title_tags,
                   topic_titles):
    
        for tag in title_tags:
            topic_titles.append(tag.text)
    
    

In [79]:
#Block-6
#A function, which consists of a bunch of sub functions.

def extraction_of_contents(topic_title_tags,topic_titles,
                          topic_disc_tags,topic_desc,
                          topic_url_tags,topic_url):
    
    
    extract_titles(topic_title_tags,topic_titles)
    extract_description(topic_disc_tags,topic_desc)
    extract_url(topic_url_tags,topic_url)
    

In [80]:
#Block-5
#This function takes a parsed html document and returns tags in a list which consists information in them.
def extract_url_tags(doc):
    
    tags = doc.find_all('a',{'class':'no-underline flex-grow-0'})
    return tags

In [81]:
#Block-4
#This function takes a parsed html document and returns tags in a list which consists information in them.
def extract_desc_tags(doc):
    
    tags = doc.find_all('p',{'class':'f5 color-fg-muted mb-0 mt-1'})
    return tags
    

In [82]:
#Block-3
#This function takes a parsed html document and returns tags in a list which consists information in them.
def extract_title_tags(doc):
    
    tags = doc.find_all('p',{'class':'f3 lh-condensed mb-0 mt-1 Link--primary'})
    return tags
    

In [83]:
#BLock-2
# takes in url,and returns a parsed html document
def page_contents(url):
    
    response = requests.get(url)
    page_content = response.text
    doc = BeautifulSoup(page_content,'html.parser')
    return doc
    

In [84]:
#Block-1
def scrape_topics_content():
    
    topic_titles = []
    topic_desc = []
    topic_url = []
    for i in range(1,5):
    
        url = 'https://github.com/topics?page='+str(i)
        doc = page_contents(url)
    
        topic_title_tags = extract_title_tags(doc)
        topic_disc_tags = extract_desc_tags(doc)
        topic_url_tags =extract_url_tags(doc) 
        
        extraction_of_contents(topic_title_tags,topic_titles,
                              topic_disc_tags,topic_desc,
                              topic_url_tags,topic_url)
    
            
    df = convert_to_dataframe(topic_titles,topic_desc,topic_url)
    df.to_csv('github_topics.csv',index = None)
    return df

In [ ]:
df = scrape_topics_content()
for i,j in df.iterrows():
    start_extraction(j[2],j[0])

# Part-2 Scrapes Each Topic's webpages upto 5.


In [ ]:
#takes in lists of usernames,reponames and stars and returns a data frame
def convert_to_DataFrame(user_names,repo_names,
                        stars):
    
    df_repo_dict = {
        'repo_user_name' : user_names,
        'repo_name' : repo_names,
        'stars' : stars
    }
    df = pd.DataFrame(df_repo_dict)
    return df

In [ ]:
#stage-2,Block-7
#Extracts stars from star tags and returns in int type
def extract_stars_count(star):
    
    if star[-1] == 'k':
        return int(float(star[:-1])*1000)
    return int(star)

In [ ]:
#stage-2,Block-6
#Extracts username and reponame from repo_tags and returns as a tuple
def extract_user_repo_names(repo_tags):
    
    user_name = repo_tags[0].text.strip()
    repo_name = repo_tags[1].text.strip()
    
    return user_name,repo_name

In [ ]:
#stage-2,Block-5
def extract_repo_info(repo_tags,repo_star_tags,
                      user_names,repo_names,
                      stars):
    
    
    for i in range(len(repo_tags)):
        
        repo_username_tags = repo_tags[i].find_all('a')
        star_count = extract_stars_count(repo_star_tags[i].text.strip())
        user_name,repo_name = extract_user_repo_names(repo_username_tags)
        
        user_names.append(user_name)
        repo_names.append(repo_name)
        stars.append(star_count)
    

In [ ]:
#stage-2,Block-4
#takes a parsed document and returns list of tags which contains star tags
def extract_star_tags(repo_contents):
    
    repo_star_tags = repo_contents.find_all('span',{'class':'Counter js-social-count'})
    return repo_star_tags

In [ ]:
#stage-2,Block-3
#takes a parsed document and returns repo tags which contains usernames and repo names
def extract_repo_tags(repo_contents):
    
        h3_selection_class = 'f3 color-fg-muted text-normal lh-condensed'
        repo_tags = repo_contents.find_all('h3',{'class':h3_selection_class})
        return repo_tags
    

In [ ]:
#stage-2,Block-2
#takes a url and returns a parsed document
def repo_doc(url):
    
    response = requests.get(url+'?page=',i)
    repo_page = response.text
    repo_contents = BeautifulSoup(repo_page,'html.parser')
    return repo_contents
    
    

In [ ]:
#stage-2 Block-1
#A main function of stage-2
def start_extraction(url,title):
    user_names = []
    repo_names = []
    stars = []
    for i in range(1,5):
        
        url = url + '?page' +str(i)
        repo_contents = repo_doc(url)
        
        repo_tags = extract_repo_tags(repo_contents)
        star_tags = extract_star_tags(repo_contents)

        extract_repo_info(repo_tags,star_tags,
                          user_names,repo_names,
                          stars)
        
    print('Extracting for',title)
    df = convert_to_DataFrame(user_names,repo_names,
                           stars)
    
    df.to_csv(title+'.csv',index = None)
    
        
        